In [1]:
import os
import pandas as pd
import numpy as np
import json
import tensorflow as tf
import matplotlib.pyplot as plt
import functions
from functions import extract_data, count_notes_per_patient, logger, count_words_per_patient, find_frequent_word, find_cooc_per_patient
from functions import cooc_log_odd_score, sequence2vec, other_emb
from functions import create_graphs_lists, train_model
from nltk.stem import PorterStemmer
from sklearn import model_selection

In [2]:
# Input vars --->
disease_name = 'PNEUMONIA'
database_path = '../MIMIC-III'
inputs_path = os.path.join('data/inputs/', disease_name)

patient_id_to_num_notes = {}

number_of_patients = {}
note_appearance_counter = {}
## Step 4
n_fold = float(3)
threshold = float(0.01)
frequent_word_lists = {}

min_sup = 0.15
# Input vars ---<

In [3]:
# Read json
with open(os.path.join(inputs_path,'patient_node_0.json'), 'r') as fp:
    patient_node_0 = json.load(fp)
with open(os.path.join(inputs_path,'patient_node_1.json'), 'r') as fp:
    patient_node_1 = json.load(fp)
# Read txt with tuples
dic = ''
with open(os.path.join(inputs_path,'patient_cooc_0.txt'),'r') as f:
         for i in f.readlines():
            dic=i #string
patient_cooc_0 = eval(dic) # this is orignal dict with instace dict

dic = ''
with open(os.path.join(inputs_path,'patient_cooc_1.txt'),'r') as f:
         for i in f.readlines():
            dic=i #string
patient_cooc_1 = eval(dic) # this is orignal dict with instace dict

# Step 6
logger.info("Get and normalize weights in co-occurrences...")
patient_cooc_set, normalized_cooc_odd_scores = cooc_log_odd_score(patient_cooc_0, patient_cooc_1, )


2021-01-15 17:03:31,543 - Get and normalize weights in co-occurrences...


In [4]:
sequence2vec = np.load(os.path.join(inputs_path,'sequence2vec.npy'), allow_pickle=True)

In [6]:
# Step 8
# Create graphs, graph labels, train and test data
logger.info("Create graphs, graph labels, train and test data...")
graphs, graph_labels, train_index, test_index = create_graphs_lists(patient_cooc_0, patient_cooc_1, normalized_cooc_odd_scores, sequence2vec)


2021-01-15 17:08:11,781 - Create graphs, graph labels, train and test data...


44.0
162


In [7]:
graph_labels

,1
0,0
1,0
2,0
3,0
4,0
...,...
883,1
884,1
885,1
886,1


In [13]:
new_test_index = np.concatenate((train_index, test_index))

In [17]:
test_index

array([178, 182,  33,  30,  19,  26, 165, 187,  15, 185,  42,   1, 195,
       179, 175,  37, 186,  36, 171, 164, 192,  20, 202,   8,  41, 198,
        40, 177,  32, 174,  39, 205,  43, 196,  12,  23,  18, 189, 169,
       168,   7,  34, 190,   2,  16,  38, 166,  22, 180,  24,   5,   6,
       184, 181, 199,   9, 206,  21, 191, 163,  10, 170, 167,   0, 203,
       176, 201, 172,  25, 193, 183,  29,  27, 197, 204,   4, 173, 194,
        11, 188,  13,  17,  28,  31,  35,   3,  14, 200])

In [14]:
new_test_index

array([820, 462, 661, 578, 735, 619, 446, 737, 538, 861, 583, 732, 876,
       425,  68, 879, 265, 466, 428, 149, 486, 132, 703, 509, 288, 289,
       577, 319, 327, 464, 108, 822, 818, 317, 632, 700, 598, 324, 719,
       767, 309, 532, 587, 819, 417, 368, 390, 279, 489, 404, 122, 300,
       762, 471, 824, 479, 744, 838, 389, 238, 626,  80, 690, 273, 260,
       268, 136,  53, 445,  94, 551, 118, 754,  57, 826, 262, 857, 627,
       490, 539, 365, 353, 447, 663, 408, 716, 320, 870, 522, 444, 791,
       595, 301, 728, 777, 335,  52, 442,  72, 835, 734, 485, 582, 611,
       413,  54, 675, 473, 510, 352, 693, 261, 492, 725, 741, 350, 714,
       102, 671, 565, 860,  77, 525,  67, 610, 154,  61, 286, 775, 695,
       569, 678, 665, 250,  86, 814, 351, 236, 670, 812, 638, 789, 519,
       568, 399, 502, 225, 592, 457, 813, 505, 677, 608, 602, 329,  49,
       237, 649, 360, 474, 623, 808, 207, 497, 128, 684, 755,  60, 590,
        83, 325, 394, 469, 381, 615, 226, 783, 129, 666, 766, 80

In [18]:
test_accs, test_f1_score, test_precision, test_recall, test_auc = train_model(graphs, graph_labels, train_index, new_test_index, "seq2vec", disease_name)

In [20]:
print(f"test_accs: {test_accs}, test_f1_score: {test_f1_score}, test_precision: {test_precision}, test_recall: {test_recall}, test_auc: {test_auc}")

test_accs: 0.920045018196106, test_f1_score: 0.9455599188804626, test_precision: 0.9034280180931091, test_recall: 0.9986111521720886, test_auc: 0.9530362486839294
